In [2]:
import mne 
from glob import glob 
import os

In [3]:
matplotlib.use('Qt5Agg') 

In [4]:
#quanti soggetti ho nella mia cartella?
subs_file_path=glob("D:\DATASET PRINCIPALE\Cartella con file EEG primi 2 subjects/*.vhdr")
print(len(subs_file_path))

2


In [6]:
#Creo la funzione che legge e processa i dati 
def prep_data(file_path):
    data= mne.io.read_raw_brainvision(file_path, preload=True)
    data.set_eeg_reference();
    data.filter(l_freq=0.1,h_freq=100)
    data.resample(250)
    return data

In [7]:
#eseguo il preprocessing di tutti i soggetti
prep_data_sub_vector=[prep_data(i)for i in subs_file_path]

Extracting parameters from D:\DATASET PRINCIPALE\Cartella con file EEG primi 2 subjects\sub-01_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3035739  =      0.000 ...  3035.739 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 33001 samples (33.001 sec)

Extracting parameters from D:\DATASET PRINCIPALE\Cartella con file EEG primi 2 subjects\sub-02_task-rsvp_eeg.vhd

In [9]:
ica=mne.preprocessing.ICA(n_components=30,random_state=0, method='fastica')

<ipython-input-9-1783480f1922>:1: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica=mne.preprocessing.ICA(n_components=30,random_state=0, method='fastica')


In [11]:
ica.plot_components(outlines="skirt")

Channels marked as bad: none


[<MNEFigure size 1950x1784 with 20 Axes>,
 <MNEFigure size 1950x992 with 10 Axes>]

In [10]:
array.shape #nepochs,nchannel, n_time_points

(283, 63, 1011)

In [11]:
from kymatio.numpy import Scattering1D
from kymatio.datasets import fetch_fsdd

In [12]:
x=array[1,1,:]

In [14]:
# Once the recording is in memory, we normalize it.
import numpy as np
x = x / np.max(np.abs(x))

In [15]:
T = x.shape[-1]
J = 6
Q = 16

In [16]:
scattering = Scattering1D(J, T, Q)

In [17]:
Sx = scattering(x)

In [18]:
meta = scattering.meta()
order0 = np.where(meta['order'] == 0)
order1 = np.where(meta['order'] == 1)
order2 = np.where(meta['order'] == 2)

In [26]:
import matplotlib.pyplot as plt


In [27]:
plt.figure(figsize=(8, 2))
plt.plot(x)
plt.title('Original signal')

plt.figure(figsize=(8, 8))
plt.subplot(3, 1, 1)
plt.plot(Sx[order0][0])
plt.title('Zeroth-order scattering')


plt.subplot(3, 1, 2)
plt.imshow(Sx[order1], aspect='auto')
plt.title('First-order scattering')


plt.subplot(3, 1, 3)
plt.imshow(Sx[order2], aspect='auto')
plt.title('Second-order scattering')

plt.show()